# Тема проекта
-
# Какие данные вы взяли для работы, укажите их источник
Я взяла набор данных, который предлагает всесторонний обзор лучших аниме 2024 года и полезен для создания систем рекомендаций, визуализации тенденций популярности аниме и их оценок, прогнозирования оценок и популярности.
[Источник](https://www.kaggle.com/datasets/bhavyadhingra00020/top-anime-dataset-2024)
# Описание каждого признака:
1. _Score_ - рейтинг или оценка, присвоенная каждому названию аниме
2. _Popularity_ - место в рейтинге каждого аниме по популярности среди зрителей
3. _Rank_ - рейтинг каждого названия аниме в наборе данных
4. _Members_ - количество участников или зрителей, связанных с каждым аниме.
5. _Description_ - краткий обзор сюжета и тем каждого аниме
6. _Synonyms_ - альтернативные названия (синонимы), используемые для каждого аниме
7. _Japanese_ - оригинальное название аниме на японском языке
8. _English_ - переведенное на английский название аниме
9. _Type_ - классификация по типу аниме (например, телесериал, фильм, OVA и т.д.)
10. _Episodes_ - общее количество серий в каждом аниме-сериале
11. _Status_ - текущее состояние аниме (например, продолжается, завершено и т.д.)
12. _Aired_ - диапазон дат выхода аниме в эфир
13. _Premiered_ - дата первой премьеры аниме
14. _Broadcast_ - информация о платформе или канале вещания
15. _Producers_ - компании или студии, участвующие в производстве аниме
16. _Licensors_ - организации или компании, владеющие лицензионными правами на аниме.
17. _Studios_ - анимационные студии, ответственные за производство аниме
18. _Source_ - исходный материал для аниме (например, манга, роман, оригинал)
19. _Genres_ - категории или жанры, к которым относится аниме
20. _Demographic_ - целевая демографическая аудитория аниме (например, SeinenSeinen, ShoujoShoujo, ShounenShounen, JoseiJosei).
21. _Duration_ - продолжительность каждого эпизода или фильма
22. _Rating_ - возрастной рейтинг контента, присвоенный каждому аниме (например, G, PG, PG-13, R)

In [ ]:
# Connecting all used libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [ ]:
# A function to create a new column ("Duration_in_minutes") based on the columns "Duration" and "Episodes"
def convert_to_minutes(row) -> int:
    duration = row['Duration']
    number_of_episodes = row['Episodes']
    time_parts = duration.split()
    if 'hr.' in duration:
        hours = int(time_parts[0])
        if 'min.' in duration:
            minutes = int(time_parts[2])
        else:
            minutes = 0
    else:
        hours = 0
        minutes = int(time_parts[0])
    return int(number_of_episodes * (hours * 60 + minutes))

In [ ]:
# Preparing of the Pandas output and loading the dataset
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 250)
df = pd.read_csv("db_anime.csv")

# EDA. Разведочный анализ данных

In [ ]:
# EDA. 1. How many rows and columns are there in the dataset?
rows, columns = df.shape
print(f'The dataset has {rows} rows and {columns} columns.')

In [ ]:
# EDA. 2. Find out the data type of each feature
data_types = df.dtypes
print("Data types of each feature:\n", data_types)

# EDA. 3. Определите тип данных для каждого объекта с точки зрения анализа
- _Непрерывный_: Score, Duration;
- _Дискретный_: Popularity, Rank, Members, Episodes;
- _Номинальный_: Description, Synonyms, Japanese, English, Type, Status, Aired, Broadcast, Producers, Licensors, Studios, Source, Genres, Demographic;
- _Порядковый_: Premiered, Rating;

In [ ]:
# EDA. 4. Are there gaps in the data?
missing_data = df.isnull().sum()
print("Missing data in each column:\n", missing_data)

# Handling missing data
# For numeric columns, we can fill missing values with the median (because it is numeric columns).
numeric_columns = ['Score', 'Popularity', 'Rank', 'Members', 'Episodes']
for column in numeric_columns:
    if df[column].isnull().sum() != 0:
        df[column].fillna(df[column].median(), inplace=True)
# For categorical columns, we can fill missing values with the mode.
categorical_columns = ['Description', 'Synonyms', 'Japanese', 'English', 'Type', 'Status', 'Aired', 'Premiered',
                       'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres', 'Demographic', 'Duration',
                       'Rating']
for column in categorical_columns:
    if df[column].isnull().sum() != 0:
        df[column].fillna(df[column].mode()[0], inplace=True)

# Verify missing data handling
missing_data_after = df.isnull().sum()
print("Missing data after handling:\n", missing_data_after)

# Creating of new feature "Duration_in_minutes"
df['Duration_in_minutes'] = df.apply(convert_to_minutes, axis=1)
print(df['Duration_in_minutes'])

# Почему я использовала именно такие способы заполнения пропусков?
При работе с пропусками стратегия заполнения этих пропусков зависит от типа данных в столбце.
- __Числовые столбцы__: Для числовых столбцов я использую медиану для заполнения пропусков. Причина этого в том, что медиана менее чувствительна к выбросам или искаженным данным по сравнению со средним значением. Если у меня есть набор данных с несколькими чрезвычайно высокими или низкими значениями, это может существенно повлиять на среднее значение, но не так сильно на медиану. Поэтому заполнение пропущенных значений медианой часто может привести к более правильному вычислению.
- __Категориальные столбцы__: Для категориальных столбцов я использую моду для заполнения пропусков. Дело в том, что в таких столбцах нет “среднего” значения, как в числовых. Использование этого способа является простой и часто эффективной стратегией для заполнения пропусков в категориальных признаках, поскольку он представляет собой наиболее распространенный признак.

In [ ]:
# EDA. 5. Are there outliers in the data?
# We'll use the IQR method to detect outliers

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers


# Detect outliers in numeric columns
numeric_columns.append('Duration_in_minutes')
outliers = {}
for column in numeric_columns:
    outliers[column] = detect_outliers(df, column)
    print(f'Outliers in {column}:\n', outliers[column])

# Removing outliers
for column in numeric_columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[~((df[column] < lower_bound) | (df[column] > upper_bound))]

# С помощью каких критериев я определяю выбросы?

Я использую метод межквартильного разбиения (IQR). Это обычный статистический метод выявления отклонений с помощью вычисления IQR. IQR - это диапазон между первым и третьим квартилем данных (про квартили я объясняю ниже). Другими словами, IQR - это диапазон средних 50% данных.
> Первый квартиль (25%) - это значение, <u>ниже</u> которого находится <u>25%</u> отсортированных по данному признаку данных. Грубо говоря, значение, стоящее на границе между первой и второй четвертью отсортированных данных. К примеру, первый квартиль рейтинга студентов ВШЭ - 75 процентиль, потому что ниже данной отметки находится четверть всех студентов.
> Третий квартиль (75%) - это значение, <u>ниже</u> которого находится <u>75%</u> отсортированных по данному признаку данных. Возвращаясь к примеру с рейтингом студентов ВШЭ, это будет уже 25 процентиль, потому что ниже этой отметки находится 75% студентов.

Теперь о том, как я использую метод IQR для выявления отклонений от нормы.
1. Определяю нижнюю (lower_bound) и верхнюю (upper_bound) границу. Для вычисления нижней - вычитаю из первого квартиля (Q1) 1.5 * IQR, для вычисления верхней - к третьему квартилю (Q3) я прибавляю 1.5 * IQR.
2. Беру все значения, которые меньше нижней границы __или__ больше верхней. Это и будут наши выбросы. Для логического "или" я использую унарный оператор "|".

In [ ]:
# EDA. 6. Calculate descriptive statistics for variables
descriptive_stats = df.describe()
print("Descriptive statistics:\n", descriptive_stats)

In [ ]:
# EDA. 7. Interpret descriptive statistics for one numeric attribute and for one categorical one.
# Numeric attribute: Score
print("Descriptive statistics for 'Score':\n", descriptive_stats['Score'])

# Categorical attribute: Type
type_counts = df['Type'].value_counts()
print("Descriptive statistics for 'Type':\n", type_counts)

# Visualization. Визуализация. Постройте не менее трех графиков (причем, нужно использовать как минимум два типа визуализаций)

In [ ]:
# Visualization. 1. Boxplot of Scores
plt.figure(figsize=(10, 5))
plt.boxplot(df['Score'], vert=False, patch_artist=True, medianprops={'linewidth': 3})
plt.title('Boxplot of Scores')
plt.xlabel('Score')
plt.grid(True)
plt.show()

## Вывод по визуализации 1
.

In [ ]:
# Visualization. 2. Bar plot of Type counts
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Type')
plt.title('Count of Anime Types')
plt.xlabel('Type')
plt.ylabel('Count')
plt.show()

## Вывод по визуализации 2
.

In [ ]:
# Visualization. 3. Scatter plot of Score vs Popularity
plt.figure(figsize=(10, 5))
# Add a regression line
sns.regplot(data=df, x='Score', y='Popularity', scatter_kws={'alpha':0.5})
plt.title('Score vs Popularity')
plt.xlabel('Score')
plt.ylabel('Popularity')
plt.show()

## Вывод по визуализации 3
.

In [ ]:
# 8. Build a correlation matrix for quantitative variables
correlation_matrix = df[numeric_columns].corr()
print("Correlation matrix:\n", correlation_matrix)
plt.figure(figsize=(12, 5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# 9. Implement 3 of the proposed 5 hypotheses

# 9.1 Perform a z-test for mathematical expectation
# Hypothesis: The average score of an anime is 7.5
mean_score = df['Score'].mean()
std_score = df['Score'].std()
n = len(df['Score'])
z_score = (mean_score - 7.5) / (std_score / np.sqrt(n))
p_value = stats.norm.sf(abs(z_score)) * 2  # two-tailed test
print(f'Hypothesis: The average score of an anime is 7.5\n'
      f'    Z-test: z_score = {z_score}, p_value = {p_value}')

# 9.2 Take a t-test for mathematical expectation
# Hypothesis: The average popularity of an anime is different from 5000
t_stat, p_value = stats.ttest_1samp(df['Popularity'], 5000)
print(f'Hypothesis: The average popularity of an anime is different from 5000\n'
      f'    T-test: t_stat = {t_stat}, p_value = {p_value}')

# 9.3 Perform a test for the equality of mathematical expectations of two samples
# Hypothesis: The average score of TV series is different from movies
tv_scores = df[df['Type'] == 'TV']['Score']
movie_scores = df[df['Type'] == 'Movie']['Score']
t_stat, p_value = stats.ttest_ind(tv_scores, movie_scores)
print(f'Hypothesis: The average score of TV series is different from movies\n'
      f'    Test for equality of means: t_stat = {t_stat}, p_value = {p_value}')

In [ ]:
# 10. Build a linear or logistic regression of at least 3 features
# Predicting Score based on Popularity, Members, and Duration_in_minutes
X = df[['Popularity', 'Members', 'Duration_in_minutes']]
y = df['Score']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
model = LinearRegression()
model.fit(X_train, y_train)

# 11. Evaluate the quality of the model
y_pred = model.predict(X_test)

# Metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'Mean Absolute Error: {mae}')